In [1]:
import server.tba as tba
import server.database as database
import pandas as pd
import numpy as np
import random

matches = tba.get_matches('2022wabon')
teams = tba.get_teams('2022wabon')

con=database.get_con()

measures=pd.read_sql('SELECT * FROM Measures;', con)

#calculates the average upper hub shots scored per team and adds it to the upper_averages dictionary
upper_averages={}
for i in teams:
    modified_df = measures[(measures['team_number']==i['team_number'])&(measures['task']=='upper')&
                           (measures['phase']=='tele')]
    nums = modified_df['measure1'].astype('int')
    average = round(nums.mean(), 3)
    upper_averages[i['team_number']] = average

#Predicts the number of upper hub shots scored per team per match, multiplies it by two for points,
#and adds it to the upper_hub dictionary
#example format of upperhub: {'qm1':{'red':[5,6,7],'blue':[1,8,3]}}
upperhub = {}
for i in matches:
    if not i['match'].startswith('qm'):
        break
    if i['match'] not in upperhub.keys():
        upperhub[i['match']]={'red':[],'blue':[]}
        
    points = np.random.poisson(upper_averages[i['team_number']])*2
    if i['alliance']=='blue':
        upperhub[i['match']]['blue'].append(points)
    elif i['alliance']=='red':
        upperhub[i['match']]['red'].append(points)

#repeats the process in the first cell for lower hub shots
lower_averages = {}
for i in teams:
    modified_df = measures[(measures['team_number']==i['team_number'])&(measures['task']=='lower')&
                          (measures['phase']=='tele')]
    nums = modified_df['measure1'].astype('int')
    lower_mean = round(nums.mean(), 3)
    lower_averages[i['team_number']] = lower_mean

#repeats the proccess in the secind cell for lower hub shots, except does not multiply by two for points
lowerhub = {}
for match in matches:
    if not match['match'].startswith('qm'):
        break
    if match['match'] not in lowerhub.keys():
        lowerhub[match['match']] = {'red':[], 'blue': []}
    points = np.random.poisson(lower_averages[match['team_number']])
    if match['alliance'] == 'red':
        lowerhub[match['match']]['red'].append(points)
    elif match['alliance'] == 'blue':
        lowerhub[match['match']]['blue'].append(points)

#gets a list of all the hangar end levels for each time, smooths the data, and stores the team number and list
#as a key value pair in hangar
hangar = {}
for team in teams:
    modified_df = measures[(measures['team_number']==team['team_number'])&(measures['task']=='hangar_level_end')&
                           (measures['phase']=='endgame')]
    end_levels = list(modified_df['measure1'].astype('int'))
    if 3 in end_levels or 4 in end_levels:
        end_levels.extend([0,1,2,3,4])
    elif 2 in end_levels or 1 in end_levels:
        end_levels.extend([0,1,2])
    hangar[team['team_number']]=end_levels

#chooses a random value for each team from their list for each match they play in
#and adds this value to hangar predictions, which is formatted similar to upperhub and lowerhub
hangar_predictions = {}
for i in matches:
    if not i['match'].startswith('qm'):
        break
    if i['match'] not in hangar_predictions.keys():
        hangar_predictions[i['match']]={'red':[],'blue':[]}
    end = random.choice(hangar[i['team_number']])
    if end == 0:
        points = 0
    elif end == 1:
        points = 4
    elif end == 2:
        points = 6
    elif end == 3:
        points = 10
    elif end == 4:
        points = 15
    if i['alliance']=='blue':
        hangar_predictions[i['match']]['blue'].append(points)
    elif i['alliance']=='red':
        hangar_predictions[i['match']]['red'].append(points)

#calculates the percentage of time each team taxied and uses that percentage to produce a random value 
#for whether or not the team taxied in each of their matches
taxi = {}
for i in teams:
    modified_df = measures[(measures['team_number']==i['team_number'])&(measures['task']=='taxi')&
                           (measures['phase']=='auto')]
    past = list(modified_df['measure1'])
    taxi[i['team_number']] = round(past.count('true')/len(past),2)

def return_num(probability):
    num = random.randint(1,100)
    if probability*100>=num:
        return 5
    else:
        return 0

#If the team taxied, it adds 5 points for the team to taxi_predictions, which is formatted the same as upperhub,
#lowerhub, and hangar_predictions
taxi_predictions = {}
for match in matches:
    if not match['match'].startswith('qm'):
        break
    if match['match'] not in taxi_predictions.keys():
        taxi_predictions[match['match']]={'red':[],'blue':[]}
    if match['alliance']=='blue':
        taxi_predictions[match['match']]['blue'].append(return_num(taxi[match['team_number']]))
    elif match['alliance']=='red':
        taxi_predictions[match['match']]['red'].append(return_num(taxi[match['team_number']]))

#Created an empty dictionary for the total points per alliance per match, formatted {{'qm1':{'red':0,'blue':0}}}
totals = {}
for match in matches:
    if not match['match'].startswith('qm'):
        break
    if match['match'] not in totals.keys():
        totals[match['match']] = {'red': 0, 'blue': 0}
#Sums all the scores in the dictionaries containing predictions for each task and adds it to the totals dictionary
for task in [upperhub, lowerhub, taxi_predictions, hangar_predictions]:
        for match, scores in task.items():
            for alliance, points in scores.items():
                task[match][alliance] = sum(points)
                totals[match][alliance] += task[match][alliance]

#diciotnary with ranking points stored as 0 for each team until we import from TBA
ranking_points={}
for team in teams:
    ranking_points[team['team_number']] = 0

#If the hangar points for an alliance in a given match were equal to or more than 16, it gives each team in the alliance
#a ranking point
for match_num, results in hangar_predictions.items():
    for alliance, score in results.items():
        if score>=16:
            for match in matches:
                if match['match']==match_num and match['alliance'] == alliance:
                    ranking_points[match['team_number']]+=1

#unrelated code to test the accuracy of TBA's predictions
predictions = tba.get_rankings('2022wabon')['match_predictions']['qual']
preds = {}
for match in matches:
    if not match['match'].startswith('qm'):
        continue
    ind = '2022wabon_'+match['match']
    preds[ind] = predictions[ind]['winning_alliance']

winners = tba.get_winners('2022wabon')
actuals = {}
for match in winners:
    if not match['key'].startswith('2022wabon_qm'):
        continue
    actuals[match['key']] = match['winning_alliance']

wrong = 0
for match in preds.keys():
    if preds[match]!=actuals[match]:
        wrong+=1
print(wrong)